In [ ]:
# Load the Boston housing dataset
from keras.datasets import boston_housing

(train_data, train_targets),(test_data, test_targets) = boston_housing.load_data()

In [ ]:
# Look at the data - 13 features
train_data.shape

In [ ]:
train_data

In [ ]:
test_data.shape

In [ ]:
# Output is housing price in thousands of dollars (these are 1970s prices unadjusted for inflation)
train_targets

In [ ]:
# Normalize the data for faster training - for each feature,
# subtract the mean of the feature and divide by the standard
# deviation, so each feature is centered around 0 with a unit
# standard deviation.
mean = train_data.mean(axis=0)
train_data = train_data - mean

std = train_data.std(axis=0)
train_data = train_data / std

# Note we use mean and std computed on the training data - don't
# use values computed on test data
test_data -= mean
test_data /= std

In [ ]:
train_data

In [ ]:
from keras import models
from keras import layers

def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu',input_shape=(train_data.shape[1],)))
    # Single layer output, no activation function, used when outputting a continuous
    # value (a price prediction in this case)
    model.add(layers.Dense(1))
    # Loss function is mean squared error, mean absolute error metric
    model.compile(optimizer='rmsprop', loss='mse',metrics=['mae'])
    return model

In [ ]:
# Not a lot of data, so use k-fold validation
import numpy as np

k = 4
num_val_samples = len(train_data) // k
num_epochs = 500
all_mae_histories = []

for i in range(k):
    print ('processing fold #', i)
    val_data = train_data[i * num_val_samples: (i+1)*num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i+1)*num_val_samples]
    
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
        train_data[(i+1) * num_val_samples:]],
        axis=0
    )
    
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
        train_targets[(i+1) * num_val_samples:]],
        axis=0
    )
    
    model = build_model()
    history = model.fit(partial_train_data, partial_train_targets,
             epochs=num_epochs, batch_size=1, verbose=0)
    mae_history = history.history['mean_absolute_error']
    all_mae_histories.append(mae_history)

In [ ]:
average_mae_history = [np.mean([x[i] for x in all_mae_histories]) for i in range(num_epochs)]

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(1, len(average_mae_history) + 1), average_mae_history)
plt.xlabel('Epochs')
plt.ylabel('Validation MAE')
plt.show()